In [1]:
#%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
import time
import math

In [3]:
#activity labels as defined in activity_labels.txt
activity_labels = ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING', 'STAND_TO_SIT', 'SIT_TO_STAND', 'SIT_TO_LIE', 'LIE_TO_SIT', 'STAND_TO_LIE', 'LIE_TO_STAND']
#activity_labels = {k:v for k,v in enumerate(activity_labels, start=1)}
#print(activity_labels)

In [4]:
#load data
def load_data(path):
    data = pd.read_csv(path, header=None, delim_whitespace=True)
    return data.values

def load_set(path, x, y):
    data = load_data(path+x)
    labels = load_data(path+y)
    return data, labels

train_data, train_labels = load_set('HAPT Data Set/Train/', 'X_train.txt', 'y_train.txt')
test_data, test_labels = load_set('HAPT Data Set/Test/', 'X_test.txt', 'y_test.txt')

print('reshaping data...')

#reshape the data to add a features dimension (features = 1)
#https://stackoverflow.com/questions/43396572/dimension-of-shape-in-conv1d
train_data = np.expand_dims(train_data, axis=2)
test_data = np.expand_dims(test_data, axis=2)

print('adjusting labels...')
#reduce the labels by 1 to match with the activity_labels and also to start labels at 0 to 11 instead of from 1 to 12
def adjust_labels (labels):
    for i in range(len(labels)-1):
        labels[i][0] -= 1

adjust_labels(train_labels);
adjust_labels(test_labels);

reshaping data...
adjusting labels...


In [5]:
#input shape
timesteps = train_data.shape[1] #561 timesteps
features = train_data.shape[2] #1 feature

In [6]:
model = models.Sequential()
model.add(layers.Conv1D(filters=32, kernel_size=3, strides=1, activation='relu', input_shape=(timesteps,features)))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=64, kernel_size=3, strides=1, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
#model.add(layers.Dense(512, activation='relu'))
#model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(12, activation='relu'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 559, 32)           128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 279, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 277, 64)           6208      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 138, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 136, 128)          24704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 68, 128)           0         
_________________________________________________________________
dropout (Dropout)            (None, 68, 128)           0

In [7]:
# Compiling the model
model.compile(optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [8]:
test_loss,test_acc = model.evaluate(test_data, test_labels, verbose=2)

3162/3162 - 2s - loss: 2.4897 - accuracy: 0.0085


In [9]:
pred_outs = model.predict_classes(test_data)

#test if the label matches the prediction
false_pred = 0
true_pred = 0
#look at predictions for the first 25 values
pred_range = 25
for i in range(pred_range):
    if not (0 <= pred_outs[i] or pred_outs[i] <= 11):
        print('prediction out of bounds')
        break
        
    print(f'Test label: {activity_labels[test_labels[i][0]]}')
    print(f'Predicted label:{activity_labels[pred_outs[i]]}')
    
    if pred_outs[i]==test_labels[i][0]:
        print('true\n')
        true_pred += 1
    else:
        print('false\n')
        false_pred += 1
print(f'False predictions:{false_pred}')
print(f'True predictions:{true_pred}')
print(f'Prediction accuraccy for first 25 values: {true_pred/pred_range}')

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STANDING
Predicted label:LIE_TO_STAND
false

Test label: STAND_TO_SIT
Predicted label:LIE_TO_STAND
false

Test label: SITTING
Predict

In [10]:
#get time of epochs to record training time
#https://stackoverflow.com/questions/43178668/record-the-computation-time-for-each-epoch-in-keras-during-model-fit
class TimeHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
        
time_callback = TimeHistory()

#train the model
model.fit(train_data, 
          train_labels, 
          epochs=15, 
          validation_data=(test_data, test_labels),
         callbacks=[time_callback])

test_loss,test_acc = model.evaluate( test_data, test_labels, verbose=2)


Train on 7767 samples, validate on 3162 samples
Epoch 1/15
7767/7767 [==============================] - 4s 505us/sample - loss: 1.3218 - accuracy: 0.5055 - val_loss: 1.0592 - val_accuracy: 0.5825
Epoch 2/15
7767/7767 [==============================] - 3s 418us/sample - loss: 1.0529 - accuracy: 0.5867 - val_loss: 1.0654 - val_accuracy: 0.5813
Epoch 3/15
7767/7767 [==============================] - 3s 406us/sample - loss: 1.0268 - accuracy: 0.5921 - val_loss: 1.0316 - val_accuracy: 0.5904
Epoch 4/15
7767/7767 [==============================] - 3s 391us/sample - loss: 1.0104 - accuracy: 0.5986 - val_loss: 1.0228 - val_accuracy: 0.5914
Epoch 5/15
7767/7767 [==============================] - 3s 394us/sample - loss: 1.0112 - accuracy: 0.5974 - val_loss: 1.0741 - val_accuracy: 0.5781
Epoch 6/15
7767/7767 [==============================] - 3s 425us/sample - loss: 1.0037 - accuracy: 0.6000 - val_loss: 1.0276 - val_accuracy: 0.5917
Epoch 7/15
7767/7767 [==============================] - 3s 429us

In [11]:
training_time = sum(time_callback.times)
print(f"Total training time: {math.floor(training_time/1)}s {math.floor(training_time%1 * 1000)}ms {math.ceil(training_time%(1/1000)*1000)}us")

Total training time: 52s 669ms 1us
